<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro-to-SA---NLP-Cookbook" data-toc-modified-id="Intro-to-SA---NLP-Cookbook-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro to SA - NLP Cookbook</a></span></li><li><span><a href="#Adv.-SA---NLP-Cookbook" data-toc-modified-id="Adv.-SA---NLP-Cookbook-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Adv. SA - NLP Cookbook</a></span></li><li><span><a href="#NLTK-Default-Vader-vs.-TextBlob" data-toc-modified-id="NLTK-Default-Vader-vs.-TextBlob-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>NLTK Default Vader vs. TextBlob</a></span><ul class="toc-item"><li><span><a href="#Vader" data-toc-modified-id="Vader-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Vader</a></span></li><li><span><a href="#TextBlob" data-toc-modified-id="TextBlob-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>TextBlob</a></span></li><li><span><a href="#Comparison" data-toc-modified-id="Comparison-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Comparison</a></span></li></ul></li><li><span><a href="#Training-Naive-Bayes-Classifier" data-toc-modified-id="Training-Naive-Bayes-Classifier-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Training Naive Bayes Classifier</a></span></li><li><span><a href="#Vader" data-toc-modified-id="Vader-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Vader</a></span></li></ul></div>

## Intro to SA - NLP Cookbook

In [44]:
import nltk
import nltk.sentiment.sentiment_analyzer

def wordBasedSentiment():
    """results in a dictionary of input words that indicate whether given words are present in the text or not."""
    positive_words = ['love', 'hope', 'joy']
    text = 'Rainfall this year brings lot of hope and joy to Farmers.'.split() # test sentence that is tokenized on space
    analysis = nltk.sentiment.util.extract_unigram_feats(text, positive_words) 
    # extract_unigram_feats() function extracts unigram features
    print(' -- single word sentiment --')
    print(analysis) # result: {'contains(love)': False, 'contains(hope)': True, 'contains(joy)': True}

def multiWordBasedSentiment():
    """We are interested in finding if pairs of words occur together in a sentence.
    We call extract_bigram_feats() on the input sentence against the sets of words in the word_sets variable.
    The result is a dictionary that tells whether these pairs of words are present in the sentence or not."""

    word_sets = [('heavy', 'rains'), ('flood', 'bengaluru')] # defines a list of two-word tuples
    text = 'heavy rains cause flash flooding in bengaluru'.split() # test sentence that is tokenized on space
    analysis = nltk.sentiment.util.extract_bigram_feats(text, word_sets)
    print(' -- multi word sentiment --')
    print(analysis) # {'contains(heavy - rains)': True, 'contains(flood - bengaluru)': False}

def markNegativity():
    text = 'Rainfall last year did not bring joy to Farmers'.split()
    negation = nltk.sentiment.util.mark_negation(text)
    print(' -- negativity --')
    print(negation) # ['Rainfall', 'last', 'year', 'did', 'not', 'bring_NEG', 'joy_NEG', 'to_NEG', 'Farmers_NEG']
    # NEG because NOT was said before

if __name__ == '__main__':
    wordBasedSentiment()
    multiWordBasedSentiment()
    markNegativity()

 -- single word sentiment --
{'contains(love)': False, 'contains(hope)': True, 'contains(joy)': True}
 -- multi word sentiment --
{'contains(heavy - rains)': True, 'contains(flood - bengaluru)': False}
 -- negativity --
['Rainfall', 'last', 'year', 'did', 'not', 'bring_NEG', 'joy_NEG', 'to_NEG', 'Farmers_NEG']


## Adv. SA - NLP Cookbook

In [45]:
import nltk
import nltk.sentiment.util
import nltk.sentiment.sentiment_analyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def mySentimentAnalyzer():
    def score_feedback(text):
        """subfunction which takes a sentence as input 
        and returns the score for the sentence in terms of -1 negative, 0 neutral, and 1 positive."""
        positive_words = ['love', 'genuine', 'liked'] # In real-world use cases, we might use words 
        # from the corpus of a larger dictionary.
        if '_NEG' in ' '.join(nltk.sentiment.util.mark_negation(text.split())): # tokenize text, mark negative tokens 
            # and join them back into 1 sentence
            score = -1 # if any word has suffix _NEG mark the sentence as negative
        else:
            analysis = nltk.sentiment.util.extract_unigram_feats(text.split(), positive_words) 
            # if the token from text is contained in the corpus of positive words,
            # i.e. if extract_unigram returns TRUE, mark it as positive
            if True in analysis.values():
                score = 1 # positive
            else:
                score = 0 # neutral
        return score

    feedback = """I love the items in this shop, very genuine and quality is well maintained.
    I have visited this shop and had samosa, my friends liked it very much.
    ok average food in this shop.
    Fridays are very busy in this shop, do not place orders during this day."""
    print(' -- custom scorer --')
    for text in feedback.split("\n"): # split on new line
        print("score = {} for >> {}".format(score_feedback(text), text)) # return score


def advancedSentimentAnalyzer():
    sentences = [
        ':)',
        ':(',
        'She is so :(',
        'I love the way cricket is played by the champions',
        'She neither likes coffee nor tea',
    ]
    senti = SentimentIntensityAnalyzer() # creates a new object for SentimentIntensityAnalyzer() 
    # and stores the object in the variable senti.
    print(' -- built-in intensity analyser --')
    for sentence in sentences:
        print('[{}]'.format(sentence), end=' --> ')
        kvp = senti.polarity_scores(sentence)
        for k in kvp:
            print('{} = {}, '.format(k, kvp[k]), end='')
        print()

if __name__ == '__main__':
    advancedSentimentAnalyzer()
    mySentimentAnalyzer()

 -- built-in intensity analyser --
[:)] --> neg = 0.0, neu = 0.0, pos = 1.0, compound = 0.4588, 
[:(] --> neg = 1.0, neu = 0.0, pos = 0.0, compound = -0.4404, 
[She is so :(] --> neg = 0.555, neu = 0.445, pos = 0.0, compound = -0.5777, 
[I love the way cricket is played by the champions] --> neg = 0.0, neu = 0.375, pos = 0.625, compound = 0.875, 
[She neither likes coffee nor tea] --> neg = 0.318, neu = 0.682, pos = 0.0, compound = -0.3252, 
 -- custom scorer --
score = 1 for >> I love the items in this shop, very genuine and quality is well maintained.
score = 1 for >>     I have visited this shop and had samosa, my friends liked it very much.
score = 0 for >>     ok average food in this shop.
score = -1 for >>     Fridays are very busy in this shop, do not place orders during this day.


## NLTK Default Vader vs. TextBlob
### Vader

In [1]:
import nltk
dataset = ["Food is good and not too expensive. Serving is just right for adult. Ambient is nice too.",
           "Used to be good. Chicken soup was below average, bbq used to be good.",
           "Food was good, standouts were the spicy beef soup and seafood pancake! ",
           "Good office lunch or after work place to go to with a big group as they have a lot of private areas with large tables",
           "As a Korean person, it was very disappointing food quality and very pricey for what you get. I won't go back there again. ",
           "Not great quality food for the price. Average food at premium prices really.",
           "Fast service. Prices are reasonable and food is decent.",
           "Extremely long waiting time. Food is decent but definitely not worth the wait.",
           "Clean premises, tasty food. My family favourites are the clear Tom yum soup, stuffed chicken wings, chargrilled squid.",
           "really good and authentic Thai food! in particular, we loved their tom yup clear soup with sliced fish. it's so well balanced that we can have it just on its own. "
           ]

In [55]:
labels = ["+", "-", "+", "+", "--", "-", "+", "-", "+", "++"]

In [2]:
def nltk_sentiment(sentence):
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    nltk_sentiment = SentimentIntensityAnalyzer()
    score = nltk_sentiment.polarity_scores(sentence)
    return score

In [4]:
import pandas as pd
pd.set_option("max_columns", 50)
pd.set_option("display.max_colwidth", -1)
nltk_results = [nltk_sentiment(row) for row in dataset]
results_df = pd.DataFrame(nltk_results)
text_df = pd.DataFrame(dataset, columns = ['text'])
nltk_df = text_df.join(results_df)
nltk_df

,text,compound,neg,neu,pos
0,Food is good and not too expensive. Serving is just right for adult. Ambient is nice too.,0.6908,0.000,0.725,0.275
1,"Used to be good. Chicken soup was below average, bbq used to be good.",0.7003,0.000,0.674,0.326
2,"Food was good, standouts were the spicy beef soup and seafood pancake!",0.4926,0.000,0.775,0.225
3,Good office lunch or after work place to go to with a big group as they have a lot of private areas with large tables,0.4404,0.000,0.884,0.116
4,"As a Korean person, it was very disappointing food quality and very pricey for what you get. I won't go back there again.",-0.5413,0.149,0.851,0.000
5,Not great quality food for the price. Average food at premium prices really.,-0.5096,0.215,0.785,0.000
6,Fast service. Prices are reasonable and food is decent.,0.0000,0.000,1.000,0.000
7,Extremely long waiting time. Food is decent but definitely not worth the wait.,0.3718,0.121,0.665,0.215
8,"Clean premises, tasty food. My family favourites are the clear Tom yum soup, stuffed chicken wings, chargrilled squid.",0.6486,0.000,0.751,0.249
9,"really good and authentic Thai food! in particular, we loved their tom yup clear soup with sliced fish. it's so well balanced that we can have it just on its own.",0.9140,0.000,0.680,0.320


- `compound` is the definitive rating of the sentiment, while the other three columns are a more detailed view of negativity, neutrality and positivity of the review.
- the result is not always accurate, especially where row nr 2 and nr 7 are clearly negative and row 6 and 8 should be more positive than neutral.
- Perhaps the dataset is not ideal for NLTK, or the texts could be too short or too ambiguous. But again, NLTK's strength lies in the trainable NaiveBayesClassifier.

### TextBlob
- The sentiment property returns a namedtuple of the form `Sentiment(polarity, subjectivity)`. 
    - The polarity score is a float within the range [-1.0, 1.0]. 
    - The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.

In [57]:
import re
from textblob import TextBlob
 
def get_sentiment(sentence):
    analysis = TextBlob(sentence)
    return(analysis.sentiment.polarity) # > 0 positive, < 0 negative
 
sa = [get_sentiment(sentence) for sentence in dataset]
sa_df = pd.DataFrame({"text":dataset, "Blob":sa, "labels":labels})
sa_df

,Blob,labels,text
0,0.237143,+,Food is good and not too expensive. Serving is just right for adult. Ambient is nice too.
1,0.162500,-,"Used to be good. Chicken soup was below average, bbq used to be good."
2,0.875000,+,"Food was good, standouts were the spicy beef soup and seafood pancake!"
3,0.228571,+,Good office lunch or after work place to go to with a big group as they have a lot of private areas with large tables
4,-0.193333,--,"As a Korean person, it was very disappointing food quality and very pricey for what you get. I won't go back there again."
5,-0.116667,-,Not great quality food for the price. Average food at premium prices really.
6,0.188889,+,Fast service. Prices are reasonable and food is decent.
7,-0.011111,-,Extremely long waiting time. Food is decent but definitely not worth the wait.
8,-0.044444,+,"Clean premises, tasty food. My family favourites are the clear Tom yum soup, stuffed chicken wings, chargrilled squid."
9,0.481944,++,"really good and authentic Thai food! in particular, we loved their tom yup clear soup with sliced fish. it's so well balanced that we can have it just on its own."


- The textblob.sentiments module contains two sentiment analysis implementations, `PatternAnalyzer` (default, based on the pattern library) and `NaiveBayesAnalyzer` (an NLTK classifier trained on a movie reviews corpus).
- NaiveBayesAnalyzer returns its result as a named tuple of the form: `Sentiment(classification, p_pos, p_neg)`.

In [58]:
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

def get_sentiment_bayes(sentence):
    analysis = TextBlob(sentence, analyzer=NaiveBayesAnalyzer()) # classifier trained on a movie reviews corpus
    return(analysis.sentiment) # > 0 positive, < 0 negative
 
sa2 = [get_sentiment_bayes(sentence) for sentence in dataset]
sa_df2 = pd.DataFrame({"text":dataset, "BlobBayes":sa2})
# pd.merge(sa_df, sa_df2, how='inner', on='text')
sa_df2.head(2)

,BlobBayes,text
0,"(pos, 0.9258831089412803, 0.07411689105872174)",Food is good and not too expensive. Serving is just right for adult. Ambient is nice too.
1,"(neg, 0.43882733819671665, 0.5611726618032842)","Used to be good. Chicken soup was below average, bbq used to be good."


### Comparison

In [61]:
dfs = [sa_df, sa_df2, nltk_df]
from functools import reduce
reduce(lambda left,right: pd.merge(left, right, on='text'), dfs) # joins 3 tables together

,Blob,labels,text,BlobBayes,compound,neg,neu,pos
0,0.237143,+,Food is good and not too expensive. Serving is just right for adult. Ambient is nice too.,"(pos, 0.9258831089412803, 0.07411689105872174)",0.6908,0.000,0.725,0.275
1,0.162500,-,"Used to be good. Chicken soup was below average, bbq used to be good.","(neg, 0.43882733819671665, 0.5611726618032842)",0.7003,0.000,0.674,0.326
2,0.875000,+,"Food was good, standouts were the spicy beef soup and seafood pancake!","(pos, 0.5012691146955478, 0.4987308853044522)",0.4926,0.000,0.775,0.225
3,0.228571,+,Good office lunch or after work place to go to with a big group as they have a lot of private areas with large tables,"(neg, 0.32801875142352493, 0.6719812485764737)",0.4404,0.000,0.884,0.116
4,-0.193333,--,"As a Korean person, it was very disappointing food quality and very pricey for what you get. I won't go back there again.","(neg, 0.2426416274342179, 0.7573583725657804)",-0.5413,0.149,0.851,0.000
5,-0.116667,-,Not great quality food for the price. Average food at premium prices really.,"(pos, 0.9509142791918158, 0.04908572080818363)",-0.5096,0.215,0.785,0.000
6,0.188889,+,Fast service. Prices are reasonable and food is decent.,"(neg, 0.3564904522147827, 0.6435095477852182)",0.0000,0.000,1.000,0.000
7,-0.011111,-,Extremely long waiting time. Food is decent but definitely not worth the wait.,"(pos, 0.5614354038358297, 0.43856459616417137)",0.3718,0.121,0.665,0.215
8,-0.044444,+,"Clean premises, tasty food. My family favourites are the clear Tom yum soup, stuffed chicken wings, chargrilled squid.","(neg, 0.4324740758643759, 0.5675259241356229)",0.6486,0.000,0.751,0.249
9,0.481944,++,"really good and authentic Thai food! in particular, we loved their tom yup clear soup with sliced fish. it's so well balanced that we can have it just on its own.","(pos, 0.898673835135922, 0.10132616486407957)",0.9140,0.000,0.680,0.320


- for this small dataset Default Blob wins!
- row nr 7 are clearly negative: Blob wins
- row 6 Blob wins
- 8 should be more positive, Vader wins
- NLTK's strength lies in the trainable NaiveBayesClassifier.

## Training Naive Bayes Classifier

In [14]:
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
n_instances = 100
subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:n_instances]]
obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:n_instances]]
len(subj_docs), len(obj_docs)
(100, 100)

(100, 100)

In [17]:
pd.DataFrame({"subj":subj_docs, "obj":obj_docs}) # Each document is represented by a tuple (sentence, label). 
# The sentence is tokenized, so it is represented by a list of strings

,obj,subj
0,"([the, movie, begins, in, the, past, where, a, young, boy, named, sam, attempts, to, save, celebi, from, a, hunter, .], obj)","([smart, and, alert, ,, thirteen, conversations, about, one, thing, is, a, small, gem, .], subj)"
1,"([emerging, from, the, human, psyche, and, showing, characteristics, of, abstract, expressionism, ,, minimalism, and, russian, constructivism, ,, graffiti, removal, has, secured, its, place, in, the, history, of, modern, art, while, being, created, by, artists, who, are, unconscious, of, their, artistic, achievements, .], obj)","([color, ,, musical, bounce, and, warm, seas, lapping, on, island, shores, ., and, just, enough, science, to, send, you, home, thinking, .], subj)"
2,"([spurning, her, mother's, insistence, that, she, get, on, with, her, life, ,, mary, is, thrown, out, of, the, house, ,, rejected, by, joe, ,, and, expelled, from, school, as, she, grows, larger, with, child, .], obj)","([it, is, not, a, mass-market, entertainment, but, an, uncompromising, attempt, by, one, artist, to, think, about, another, .], subj)"
3,"([amitabh, can't, believe, the, board, of, directors, and, his, mind, is, filled, with, revenge, and, what, better, revenge, than, robbing, the, bank, himself, ,, ironic, as, it, may, sound, .], obj)","([a, light-hearted, french, film, about, the, spiritual, quest, of, a, fashion, model, seeking, peace, of, mind, while, in, a, love, affair, with, a, veterinarian, who, is, a, non-practicing, jew, .], subj)"
4,"([she, ,, among, others, excentricities, ,, talks, to, a, small, rock, ,, gertrude, ,, like, if, she, was, alive, .], obj)","([my, wife, is, an, actress, has, its, moments, in, looking, at, the, comic, effects, of, jealousy, ., in, the, end, ,, though, ,, it, is, only, mildly, amusing, when, it, could, have, been, so, much, more, .], subj)"
5,"([this, gives, the, girls, a, fair, chance, of, pulling, the, wool, over, their, eyes, using, their, sexiness, to, poach, any, last, vestige, of, common, sense, the, dons, might, have, had, .], obj)","([works, both, as, an, engaging, drama, and, an, incisive, look, at, the, difficulties, facing, native, americans, .], subj)"
6,"([styled, after, vh1's, "", behind, the, music, ,, "", this, mockumentary, profiles, the, rise, and, fall, of, an, internet, startup, ,, called, icevan, ., com, .], obj)","([even, a, hardened, voyeur, would, require, the, patience, of, job, to, get, through, this, interminable, ,, shapeless, documentary, about, the, swinging, subculture, .], subj)"
7,"([being, blue, is, not, his, only, predicament, ;, he, also, lacks, the, ability, to, outwardly, express, his, emotions, .], obj)","([when, perry, fists, a, bull, at, the, moore, farm, ,, it's, only, a, matter, of, time, before, he, gets, the, upper, hand, in, matters, of, the, heart, .], subj)"
8,"([the, killer's, clues, are, a, perversion, of, biblical, punishments, for, sins, :, stoning, ,, burning, ,, decapitation, .], obj)","([the, characters, ., ., ., are, paper-thin, ,, and, their, personalities, undergo, radical, changes, when, it, suits, the, script, .], subj)"
9,"([david, is, a, painter, with, painter's, block, who, takes, a, job, as, a, waiter, to, get, some, inspiration, .], obj)","([the, script, is, a, tired, one, ,, with, few, moments, of, joy, rising, above, the, stale, material, .], subj)"


In [30]:
# We separately split subj. and objective instances to keep a balanced uniform class distribution in both train and test sets.
import re
train_subj_docs = subj_docs[:80]
test_subj_docs = subj_docs[80:100]
train_obj_docs = obj_docs[:80]
test_obj_docs = obj_docs[80:100]
training_docs = train_subj_docs+train_obj_docs
testing_docs = test_subj_docs+test_obj_docs

sentim_analyzer = SentimentAnalyzer()
all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in training_docs])
print(all_words_neg[:100])

['smart', 'and', 'alert', ',', 'thirteen', 'conversations', 'about', 'one', 'thing', 'is', 'a', 'small', 'gem', '.', 'color', ',', 'musical', 'bounce', 'and', 'warm', 'seas', 'lapping', 'on', 'island', 'shores', '.', 'and', 'just', 'enough', 'science', 'to', 'send', 'you', 'home', 'thinking', '.', 'it', 'is', 'not', 'a_NEG', 'mass-market_NEG', 'entertainment_NEG', 'but_NEG', 'an_NEG', 'uncompromising_NEG', 'attempt_NEG', 'by_NEG', 'one_NEG', 'artist_NEG', 'to_NEG', 'think_NEG', 'about_NEG', 'another_NEG', '.', 'a', 'light-hearted', 'french', 'film', 'about', 'the', 'spiritual', 'quest', 'of', 'a', 'fashion', 'model', 'seeking', 'peace', 'of', 'mind', 'while', 'in', 'a', 'love', 'affair', 'with', 'a', 'veterinarian', 'who', 'is', 'a', 'non-practicing', 'jew', '.', 'my', 'wife', 'is', 'an', 'actress', 'has', 'its', 'moments', 'in', 'looking', 'at', 'the', 'comic', 'effects', 'of', 'jealousy']


In [34]:
# We use simple unigram word features, handling negation:
unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)
len(unigram_feats)

83

In [35]:
print(unigram_feats[:100])

['.', 'the', ',', 'a', 'and', 'of', 'to', 'is', 'in', 'with', 'it', 'that', 'his', 'on', 'for', 'an', 'who', 'by', 'he', 'from', 'her', '"', 'film', 'as', 'this', 'movie', 'their', 'but', 'one', 'at', 'about', 'the_NEG', 'a_NEG', 'to_NEG', 'are', "there's", '(', 'story', 'when', 'so', 'be', ',_NEG', ')', 'they', 'you', 'not', 'have', 'like', 'will', 'all', 'into', 'out', 'she', 'what', 'life', 'has', 'its', 'only', 'more', 'even', '--', ':', 'can', ';', 'home', 'look', "it's", 'if', 'where', 'most', 'him', 'search', 'but_NEG', 'love', 'both', 'make', 'begins', 'some', 'two', 'of_NEG', 'made', 'which', 'them']


In [36]:
sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)

In [40]:
# We apply features to obtain a feature-value representation of our datasets:
training_set = sentim_analyzer.apply_features(training_docs)
test_set = sentim_analyzer.apply_features(testing_docs)
print(training_set[0])

({'contains(.)': True, 'contains(the)': False, 'contains(,)': True, 'contains(a)': True, 'contains(and)': True, 'contains(of)': False, 'contains(to)': False, 'contains(is)': True, 'contains(in)': False, 'contains(with)': False, 'contains(it)': False, 'contains(that)': False, 'contains(his)': False, 'contains(on)': False, 'contains(for)': False, 'contains(an)': False, 'contains(who)': False, 'contains(by)': False, 'contains(he)': False, 'contains(from)': False, 'contains(her)': False, 'contains(")': False, 'contains(film)': False, 'contains(as)': False, 'contains(this)': False, 'contains(movie)': False, 'contains(their)': False, 'contains(but)': False, 'contains(one)': True, 'contains(at)': False, 'contains(about)': True, 'contains(the_NEG)': False, 'contains(a_NEG)': False, 'contains(to_NEG)': False, 'contains(are)': False, "contains(there's)": False, 'contains(()': False, 'contains(story)': False, 'contains(when)': False, 'contains(so)': False, 'contains(be)': False, 'contains(,_NEG)'

In [41]:
from nltk.classify import NaiveBayesClassifier
# We can now train our classifier on the training set, and subsequently output the evaluation results:
trainer = NaiveBayesClassifier.train
classifier = sentim_analyzer.train(trainer, training_set)
#Training classifier
for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
     print('{0}: {1}'.format(key, value)) # Evaluating NaiveBayesClassifier results...

Training classifier
Evaluating NaiveBayesClassifier results...
Accuracy: 0.8
F-measure [obj]: 0.8
F-measure [subj]: 0.8
Precision [obj]: 0.8
Precision [subj]: 0.8
Recall [obj]: 0.8
Recall [subj]: 0.8


## Vader

In [42]:
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> sentences = ["VADER is smart, handsome, and funny.", # positive sentence example
...    "VADER is smart, handsome, and funny!", # punctuation emphasis handled correctly (sentiment intensity adjusted)
...    "VADER is very smart, handsome, and funny.",  # booster words handled correctly (sentiment intensity adjusted)
...    "VADER is VERY SMART, handsome, and FUNNY.",  # emphasis for ALLCAPS handled
...    "VADER is VERY SMART, handsome, and FUNNY!!!",# combination of signals - VADER appropriately adjusts intensity
...    "VADER is VERY SMART, really handsome, and INCREDIBLY FUNNY!!!",# booster words & punctuation make this close to ceiling for score
...    "The book was good.",         # positive sentence
...    "The book was kind of good.", # qualified positive sentence is handled correctly (intensity adjusted)
...    "The plot was good, but the characters are uncompelling and the dialog is not great.", # mixed negation sentence
...    "A really bad, horrible book.",       # negative sentence with booster words
...    "At least it isn't a horrible book.", # negated negative sentence with contraction
...    ":) and :D",     # emoticons handled
...    "",              # an empty string is correctly handled
...    "Today sux",     #  negative slang handled
...    "Today sux!",    #  negative slang with punctuation emphasis handled
...    "Today SUX!",    #  negative slang with capitalization emphasis
...    "Today kinda sux! But I'll get by, lol" # mixed sentiment example with slang and constrastive conjunction "but"
... ]
>>> paragraph = "It was one of the worst movies I've seen, despite good reviews. \
... Unbelievably bad acting!! Poor direction. VERY poor production. \
... The movie was bad. Very bad movie. VERY bad movie. VERY BAD movie. VERY BAD movie!"

>>> from nltk import tokenize
>>> lines_list = tokenize.sent_tokenize(paragraph)
>>> sentences.extend(lines_list)

In [ ]:
>>> tricky_sentences = [
...    "Most automated sentiment analysis tools are shit.",
...    "VADER sentiment analysis is the shit.",
...    "Sentiment analysis has never been good.",
...    "Sentiment analysis with VADER has never been this good.",
...    "Warren Beatty has never been so entertaining.",
...    "I won't say that the movie is astounding and I wouldn't claim that \
...    the movie is too banal either.",
...    "I like to hate Michael Bay films, but I couldn't fault this one",
...    "It's one thing to watch an Uwe Boll film, but another thing entirely \
...    to pay for it",
...    "The movie was too good",
...    "This movie was actually neither that funny, nor super witty.",
...    "This movie doesn't care about cleverness, wit or any other kind of \
...    intelligent humor.",
...    "Those who find ugly meanings in beautiful things are corrupt without \
...    being charming.",
...    "There are slow and repetitive parts, BUT it has just enough spice to \
...    keep it interesting.",
...    "The script is not fantastic, but the acting is decent and the cinematography \
...    is EXCELLENT!",
...    "Roger Dodger is one of the most compelling variations on this theme.",
...    "Roger Dodger is one of the least compelling variations on this theme.",
...    "Roger Dodger is at least compelling as a variation on the theme.",
...    "they fall in love with the product",
...    "but then it breaks",
...    "usually around the time the 90 day warranty expires",
...    "the twin towers collapsed today",
...    "However, Mr. Carter solemnly argues, his client carried out the kidnapping \
...    under orders and in the ''least offensive way possible.''"
... ]
>>> sentences.extend(tricky_sentences)

In [43]:
>>> sid = SentimentIntensityAnalyzer()
>>> for sentence in sentences:
...     print(sentence)
...     ss = sid.polarity_scores(sentence)
...     for k in sorted(ss):
...         print('{0}: {1}, '.format(k, ss[k]), end='')
...     print()

VADER is smart, handsome, and funny.
compound: 0.8316, neg: 0.0, neu: 0.254, pos: 0.746, 
VADER is smart, handsome, and funny!
compound: 0.8439, neg: 0.0, neu: 0.248, pos: 0.752, 
VADER is very smart, handsome, and funny.
compound: 0.8545, neg: 0.0, neu: 0.299, pos: 0.701, 
VADER is VERY SMART, handsome, and FUNNY.
compound: 0.9227, neg: 0.0, neu: 0.246, pos: 0.754, 
VADER is VERY SMART, handsome, and FUNNY!!!
compound: 0.9342, neg: 0.0, neu: 0.233, pos: 0.767, 
VADER is VERY SMART, really handsome, and INCREDIBLY FUNNY!!!
compound: 0.9469, neg: 0.0, neu: 0.294, pos: 0.706, 
The book was good.
compound: 0.4404, neg: 0.0, neu: 0.508, pos: 0.492, 
The book was kind of good.
compound: 0.3832, neg: 0.0, neu: 0.657, pos: 0.343, 
The plot was good, but the characters are uncompelling and the dialog is not great.
compound: -0.7042, neg: 0.327, neu: 0.579, pos: 0.094, 
A really bad, horrible book.
compound: -0.8211, neg: 0.791, neu: 0.209, pos: 0.0, 
At least it isn't a horrible book.
compound